In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.path.insert(0, './src')
%matplotlib inline


from metrics import NDCG_binary_at_k_batch, Recall_at_k_batch
from models import MultiVAE, MultiDAE, Multi_our_VAE, MultiHoffmanVAE, Multi_ourHoffman_VAE
from training import train_model
from data import Dataset
from args import get_args
import numpy as np
import pandas as pd

import pdb

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import ncvis




device = 'cpu'

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [2]:
args = dotdict({})
args.train_batch_size = 500
args.val_batch_size = 2000
args.data = 'ml20m'
args.device = device

args.learning_rate = 1e-4
args.n_epoches = 200
args.l2_coeff = 0.01 / args.train_batch_size
args.print_info_ = 1

args.annealing = True

if args.annealing:
    args.total_anneal_steps = 200000
    args.anneal_cap = 0.2
else:
    args.total_anneal_steps = 0
    args.anneal_cap = 1.

device_zero = torch.tensor(0., device=device, dtype=torch.float32)
device_one = torch.tensor(1., device=device, dtype=torch.float32)
std_normal = torch.distributions.Normal(loc=device_zero,
                                    scale=device_one)
    
dataset = Dataset(args, data_dir='./data/')

In [3]:
models = [model for model in sorted(os.listdir('./models')) if model.startswith('best') ]
for m in models:
    print('\'' + m + '\',' )

'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False.pt',
'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True.pt',
'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False.pt',
'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True.pt',
'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_False.pt',
'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_True.pt',
'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_False.pt',
'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_True.pt',
'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False.pt',
'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt',
'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_False.pt',
'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True.pt',
'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_False.pt',
'best_model_Multi_ourHoffman_VAE_K_1_N_1_lear

In [4]:
# models = [
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_True.pt',
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_True.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_False.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_True.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_True.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_False.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True.pt',
# ]

In [3]:
def load_model(path, device):
    device_zero = torch.tensor(0., device=device, dtype=torch.float32)
    device_one = torch.tensor(1., device=device, dtype=torch.float32)
    std_normal = torch.distributions.Normal(loc=device_zero,
                                        scale=device_one)
    uniform = torch.distributions.Uniform(low=device_zero, high=device_one)
    torch_log_2 = torch.tensor(np.log(2), device=device, dtype=torch.float32)
    
    model = torch.load(path, map_location=device)
    
    model.eval()
    model.std_normal = std_normal
    if hasattr(model, 'torch_log_2'):
        model.torch_log_2 = torch_log_2
    if hasattr(model, 'transitions'):
        model.transitions = model.transitions.to(device)
        if type(model.transitions) == type(torch.nn.ModuleList([])):
            for i in range(len(model.transitions)):
                model.transitions[i].device = device
                model.transitions[i].device_zero = device_zero
                model.transitions[i].device_one = device_one
        else:
            model.transitions.device = device
            model.transitions.device_zero = device_zero
            model.transitions.device_one = device_one
    if hasattr(model, 'reverse_kernel'):
        model.reverse_kernel = model.reverse_kernel.to(device)
        model.reverse_kernel.device = device
        model.device_one = device_one
        
    return model

In [6]:
resulting_dict = {'model': [], 'ndcg': [], 'recall': [], 'noise': []}

for noise in [1., 0.]:
    for m in tqdm(models):
        model = load_model(os.path.join('./models', m), device)

        ndcg = []
        recall = []
        for bnum, batch_val in enumerate(dataset.next_val_batch()):
            out = model(batch_val[0], is_training_ph=noise)  ### Here we are using noise when estimate metrics
            pred_val = out[0]
            X = batch_val[0].cpu().detach().numpy()
            pred_val = pred_val.cpu().detach().numpy()
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf
            ndcg.append(NDCG_binary_at_k_batch(pred_val, batch_val[1]))
            recall.append(Recall_at_k_batch(pred_val, batch_val[1]))

        ndcg_dist = np.concatenate(ndcg)
        recall_dist = np.concatenate(recall)
        current_ndcg = ndcg_dist.mean()
        current_recall = recall_dist.mean()
        resulting_dict['model'].append(m)
        resulting_dict['ndcg'].append(current_ndcg)
        resulting_dict['recall'].append(current_recall)
        resulting_dict['noise'].append(noise)
        print(m, 'NDCG: ', current_ndcg, '\t', 'Recall: ', current_recall)
        del model

  2%|▏         | 1/50 [00:22<18:41, 22.90s/it]

best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False.pt NDCG:  0.3599860801663479 	 Recall:  0.5913827374137361


  4%|▍         | 2/50 [00:46<18:33, 23.20s/it]

best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True.pt NDCG:  0.3869960989637224 	 Recall:  0.6260463131454891


  6%|▌         | 3/50 [01:34<23:54, 30.51s/it]

...

best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True.pt NDCG:  0.4325399914134912 	 Recall:  0.6629434271066635


 98%|█████████▊| 49/50 [30:19<01:04, 64.30s/it]

best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_False.pt NDCG:  0.3947527895847348 	 Recall:  0.6248612436214498


100%|██████████| 50/50 [31:33<00:00, 37.87s/it]

best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True.pt NDCG:  0.4341622631175437 	 Recall:  0.6630982773745904


In [7]:
pd.set_option('display.max_colwidth', 100)

In [8]:
df = pd.DataFrame(resulting_dict)
df.to_csv('./eval_results.csv', index=False)
df

,model,ndcg,recall,noise
0,best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False.pt,0.359986,0.591383,1.0
1,best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True.pt,0.386996,0.626046,1.0
2,best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False.pt,0.355633,0.584347,1.0
3,best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True.pt,0.387190,0.620263,1.0
4,best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False.pt,0.361924,0.595574,1.0
5,best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt,0.392378,0.628635,1.0
6,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_False.pt,0.358793,0.589407,1.0
7,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True.pt,0.394134,0.628383,1.0
8,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_False.pt,0.360511,0.590860,1.0
9,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_True.pt,0.388402,0.627646,1.0


In [14]:
df[df.noise == 1.].sort_values('ndcg', ascending=False).head(9)

,model,ndcg,recall,noise
17,best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True.pt,0.398662,0.635319,1.0
47,best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True.pt,0.394738,0.632062,1.0
7,best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True.pt,0.394134,0.628383,1.0
45,best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True.pt,0.393916,0.631515,1.0
49,best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True.pt,0.393346,0.627312,1.0
15,best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True.pt,0.392814,0.630674,1.0
19,best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True.pt,0.392658,0.628903,1.0
16,best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_False_anneal_True.pt,0.392455,0.629456,1.0
5,best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt,0.392378,0.628635,1.0


In [12]:
df[df.noise == 0.].sort_values('ndcg', ascending=False).head(19)

,model,ndcg,recall,noise
99,best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True.pt,0.434162,0.663098,0.0
71,best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_True.pt,0.433514,0.664621,0.0
77,best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_True.pt,0.433261,0.663767,0.0
65,best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True.pt,0.432926,0.663661,0.0
79,best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_True.pt,0.432902,0.663984,0.0
69,best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True.pt,0.432894,0.663048,0.0
87,best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_True.pt,0.432892,0.662653,0.0
89,best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_True.pt,0.432888,0.663717,0.0
85,best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_True.pt,0.432888,0.663290,0.0
60,best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_False_anneal_True.pt,0.432805,0.661502,0.0


# Parameters tuning

In [4]:
model = load_model('./models/best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False.pt', 'cpu')
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt'

In [5]:
for p in model.parameters():
    p.requires_grad_(False)
model.eval()
fixed_decoder = model.decoder
del model

layers = [200, 600, dataset.n_items]

model = MultiVAE(layers, args=args)
model.decoder = fixed_decoder

Sequential(
  (0): Linear(in_features=20108, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=400, bias=True)
)
Sequential(
  (0): Linear(in_features=200, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=20108, bias=True)
)


In [9]:
train_model(model, dataset, args)

  0%|          | 0/200 [00:00<?, ?it/s]

666.53955
592.1258


KeyboardInterrupt: 